# Hybrid HHL vs Quantum HHL: Oracle Comparison

This notebook benchmarks two eigenvalue inversion oracle strategies for the HHL algorithm:

| Label | `eig_oracle` | Description |
|---|---|---|
| **Hybrid HHL** | `"classical"` | Classically computes eigenvalues; builds exact controlled-RY rotations per QPE state |
| **Quantum HHL** | `"quantum"` | Fully quantum inversion via Qiskit's `ExactReciprocalGate` (no classical eigenvalue pre-computation) |

The analysis is split into two parts:
1. **Circuit Resources** — How depth, 2-qubit gate count, and total gate count scale with problem size $n$
2. **Solution Quality** — How residual $\|Ax - b\|$ varies with problem size, condition number, and sparsity across an ensemble of randomly generated problems

In [ ]:
from pathlib import Path
import sys

def find_repo_root(start=None):
    p = (start or Path.cwd()).resolve()
    for d in (p, *p.parents):
        if (d / ".git").exists() or (d / "pyproject.toml").exists() or (d / "src").exists():
            return d
    return p

repo_root = find_repo_root()
for p in [repo_root / "src", repo_root]:
    if str(p) not in sys.path:
        sys.path.insert(0, str(p))

print("repo root:", repo_root)

In [ ]:
from datetime import datetime
import math
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.colors import Normalize
from qiskit_aer import AerSimulator

from qlsas.qlsa.hhl.hhl import HHL
from qlsas.data_loader import StatePrep
from qlsas.transpiler import Transpiler
from qlsas.post_processor import Post_Processor
from qlsas.solver import QuantumLinearSolver
from linear_systems_problems.random_matrix_generator import generate_problem

warnings.filterwarnings("ignore")
%config InlineBackend.figure_format = "retina"

plt.rcParams.update({
    "font.family": "serif",
    "font.size": 12,
    "axes.labelsize": 13,
    "axes.titlesize": 13,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "legend.fontsize": 11,
    "figure.dpi": 130,
    "savefig.dpi": 300,
    "savefig.bbox": "tight",
})

ORACLE_STYLE = {
    "hybrid": {"color": "#2196F3", "marker": "o", "label": "Hybrid HHL (classical oracle)"},
    "quantum": {"color": "#E91E63", "marker": "s", "label": "Quantum HHL (quantum oracle)"},
}

In [ ]:
# ── Configuration ─────────────────────────────────────────────────────────────

ORACLE_MAP = {
    "hybrid":  "classical",   # eig_oracle value for HHL
    "quantum": "quantum",
}

RESOURCE_SIZES = [2, 4, 8, 16, 32, 64]   # problem dims for resource analysis (no circuit execution)
PERF_SIZES     = [2, 4, 8, 16]            # problem dims for performance runs

COND_NUMBERS = [2.0, 5.0, 10.0, 30.0]
SPARSITIES   = [0.1, 0.5, 0.9]
SEEDS        = [0, 1, 2]

# Fixed values used when sweeping a different axis
FIXED_N        = 8
FIXED_COND     = 10.0
FIXED_SPARSITY = 0.5

# Use log2(n) QPE qubits — matches the data register size
def num_qpe(n): return int(math.log2(n))

# Sampler settings
TARGET_SHOTS   = 150    # successful ancilla shots per run
SHOTS_PER_BATCH = 1500
OPT_LEVEL      = 0      # transpiler opt level for benchmarking (speed)

BACKEND = AerSimulator()

In [ ]:
# ── Helper functions ──────────────────────────────────────────────────────────

def _circuit_stats(circ):
    """Extract lightweight resource metrics from a QuantumCircuit."""
    ops = circ.count_ops()
    two_q = circ.num_nonlocal_gates()
    total = sum(ops.values())
    return {
        "depth": circ.depth(),
        "two_q_gates": two_q,
        "total_gates": total,
        "num_qubits": circ.num_qubits,
    }


def get_resources_only(oracle_key, n, seed=0, opt_level=3):
    """
    Build and transpile an HHL circuit without executing it.
    Returns raw and transpiled resource metrics.
    """
    prob = generate_problem(n=n, cond_number=FIXED_COND, sparsity=FIXED_SPARSITY, seed=seed)
    A = prob["A"]
    b = prob["b"] / np.linalg.norm(prob["b"])

    hhl = HHL(
        state_prep=StatePrep(method="default"),
        readout="measure_x",
        num_qpe_qubits=num_qpe(n),
        eig_oracle=ORACLE_MAP[oracle_key],
    )
    circ = hhl.build_circuit(A, b)

    transpiler = Transpiler(circuit=circ, backend=BACKEND, optimization_level=opt_level)
    t_circ = transpiler.optimize()

    tran = _circuit_stats(t_circ)
    return {
        "oracle": oracle_key, "n": n,
        "num_qubits":  circ.num_qubits,
        "tr_depth":    tran["depth"],
        "tr_2q_gates": tran["two_q_gates"],
    }


def run_and_profile(oracle_key, n, cond_number, sparsity, seed):
    """
    Run the full HHL workflow for one problem instance.
    Returns performance and resource metrics.
    """
    prob = generate_problem(n=n, cond_number=cond_number, sparsity=sparsity, seed=seed)
    A = prob["A"]
    b = prob["b"] / np.linalg.norm(prob["b"])

    hhl = HHL(
        state_prep=StatePrep(method="default"),
        readout="measure_x",
        num_qpe_qubits=num_qpe(n),
        eig_oracle=ORACLE_MAP[oracle_key],
    )
    solver = QuantumLinearSolver(
        qlsa=hhl,
        backend=BACKEND,
        target_successful_shots=TARGET_SHOTS,
        shots_per_batch=SHOTS_PER_BATCH,
        optimization_level=OPT_LEVEL,
    )

    try:
        solution = solver.solve(A, b, verbose=False)
        pp = Post_Processor()
        alpha = pp.norm_estimation(A, b, solution)
        residual = float(np.linalg.norm(b - A @ (alpha * solution)))

        return {
            "oracle": oracle_key, "n": n,
            "cond_number": cond_number, "sparsity": sparsity, "seed": seed,
            "residual":   residual,
            "num_qubits": solver.circuit.num_qubits,
            "success":    True,
        }
    except Exception as e:
        return {
            "oracle": oracle_key, "n": n,
            "cond_number": cond_number, "sparsity": sparsity, "seed": seed,
            "residual": np.nan, "success": False, "error": str(e),
        }

---
## Part 1 — Circuit Resource Scaling

We build and transpile circuits for both oracles across problem sizes $n \in \{2, 4, 8, 16, 32\}$ **without executing them**. This directly shows how the oracle choice affects gate complexity.

**Key difference:**
- The *Hybrid* oracle appends $2^m$ multi-controlled RY gates (one per QPE register state). After decomposition these contribute $O(2^m \cdot m)$ two-qubit gates.
- The *Quantum* oracle uses `ExactReciprocalGate` which implements $|k\rangle \to |C/\lambda_k\rangle$ as a quantum arithmetic circuit, with a different depth/width trade-off.

In [ ]:
resource_records = []

for oracle_key in ORACLE_MAP:
    for n in RESOURCE_SIZES:
        print(f"  {oracle_key:8s}  n={n:3d} ...", end="", flush=True)
        rec = get_resources_only(oracle_key, n, seed=0, opt_level=3)
        resource_records.append(rec)
        print(f"  tr depth={rec['tr_depth']:6d}  tr 2q gates={rec['tr_2q_gates']:6d}")

df_res = pd.DataFrame(resource_records)
df_res

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

metrics = [
    ("tr_depth",        "Transpiled circuit depth (opt=3)", axes[0]),
    ("tr_2q_gates",     "Transpiled 2-qubit gate count",   axes[1]),
]

for col, title, ax in metrics:
    for oracle_key, style in ORACLE_STYLE.items():
        sub = df_res[df_res["oracle"] == oracle_key].sort_values("n")
        ax.plot(sub["n"], sub[col],
                marker=style["marker"], color=style["color"],
                linewidth=2, markersize=7, label=style["label"])
    ax.set_xlabel("Problem size $n$")
    ax.set_ylabel(title)
    ax.set_title(title)
    ax.set_xticks(RESOURCE_SIZES)
    ax.set_xticklabels([str(n) for n in RESOURCE_SIZES])
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

fig.suptitle("Circuit Resource Scaling vs Problem Size", fontsize=15, fontweight="bold", y=1.01)
plt.tight_layout()
plt.savefig(f"../data/oracle_comparison_resources_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf")
plt.show()

In [ ]:
# Qubit count — same for both oracles (oracle only adds gates, not qubits)
fig, ax = plt.subplots(figsize=(6, 4))
sub = df_res[df_res["oracle"] == "hybrid"].sort_values("n")
ax.plot(sub["n"], sub["num_qubits"], "o-", color="gray", linewidth=2, markersize=8)
for _, row in sub.iterrows():
    nq = int(row["num_qubits"])
    ax.annotate(
        f"{nq} qubits",
        xy=(row["n"], nq), xytext=(4, 4), textcoords="offset points", fontsize=10
    )
ax.set_xlabel("Problem size $n$")
ax.set_ylabel("Total qubits")
ax.set_title("Total qubit count vs $n$  (same for both oracles)")
ax.set_xticks(RESOURCE_SIZES)
ax.set_xticklabels([str(n) for n in RESOURCE_SIZES])
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f"../data/oracle_comparison_qubits_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf")
plt.show()
print("Note: qubit count = 1 (ancilla) + log2(n) (QPE) + log2(n) (data) = 1 + 2*log2(n)")

In [ ]:
# Overhead ratio: how many more gates does hybrid use vs quantum?
pivot = df_res.pivot(index="n", columns="oracle", values=["tr_depth", "tr_2q_gates"])
ratio = pd.DataFrame(index=df_res["n"].unique())
for col in ["tr_depth", "tr_2q_gates"]:
    ratio[f"{col}_ratio (hybrid/quantum)"] = (
        pivot[col]["hybrid"] / pivot[col]["quantum"]
    )
ratio.sort_index(inplace=True)
ratio.index.name = "n"
print("Hybrid / Quantum resource ratios:")
ratio.round(2)

---
## Part 2 — Solution Quality Benchmark

We run the full HHL workflow on an ensemble of randomly generated Hermitian positive-definite systems and compare the residual $\|Ax - b\|_2$ (after optimal scaling) between the two oracle strategies.

Three sweeps:
1. **Condition number** — fixed $n=8$, $\text{sparsity}=0.5$, sweep $\kappa \in \{2, 5, 10, 30\}$
2. **Sparsity** — fixed $n=8$, $\kappa=10$, sweep sparsity $\in \{0.1, 0.5, 0.9\}$
3. **Problem size** — fixed $\kappa=10$, sparsity$=0.5$, sweep $n \in \{2, 4, 8\}$

Each configuration is averaged over 3 random seeds.

In [ ]:
perf_records = []
total = len(ORACLE_MAP) * (len(COND_NUMBERS) + len(SPARSITIES) + len(PERF_SIZES)) * len(SEEDS)
done = 0

def _run(oracle_key, n, cond_number, sparsity):
    global done
    for seed in SEEDS:
        rec = run_and_profile(oracle_key, n, cond_number, sparsity, seed)
        perf_records.append(rec)
        done += 1
        status = "OK" if rec["success"] else f"FAIL: {rec.get('error','')[:40]}"
        print(f"[{done:3d}/{total}] {oracle_key:7s} n={n:2d}  κ={cond_number:5.1f}  sp={sparsity:.1f}  "
              f"seed={seed}  → residual={rec.get('residual', float('nan')):.4f}  {status}")

# Sweep 1: condition number (fixed n, sparsity)
print("=" * 70)
print("Sweep 1: condition number")
print("=" * 70)
for oracle_key in ORACLE_MAP:
    for kappa in COND_NUMBERS:
        _run(oracle_key, FIXED_N, kappa, FIXED_SPARSITY)

# Sweep 2: sparsity (fixed n, cond)
print("=" * 70)
print("Sweep 2: sparsity")
print("=" * 70)
for oracle_key in ORACLE_MAP:
    for sp in SPARSITIES:
        _run(oracle_key, FIXED_N, FIXED_COND, sp)

# Sweep 3: problem size (fixed cond, sparsity)
print("=" * 70)
print("Sweep 3: problem size")
print("=" * 70)
for oracle_key in ORACLE_MAP:
    for n in PERF_SIZES:
        _run(oracle_key, n, FIXED_COND, FIXED_SPARSITY)

df_perf = pd.DataFrame(perf_records)
print(f"\nTotal records: {len(df_perf)}  |  Failed: {(~df_perf['success']).sum()}")

In [ ]:
# ── Plot helper ───────────────────────────────────────────────────────────────

def plot_residual_sweep(df, x_col, x_label, filter_col, filter_val, ax, log_y=True):
    """
    Plot mean ± std of log10(residual) vs x_col for each oracle.
    Filters df to rows where filter_col == filter_val.
    """
    sub = df[np.isclose(df[filter_col], filter_val) & df["success"]].copy()
    sub["log_residual"] = np.log10(sub["residual"].clip(lower=1e-10))

    for oracle_key, style in ORACLE_STYLE.items():
        grp = sub[sub["oracle"] == oracle_key].groupby(x_col)["log_residual"]
        means = grp.mean()
        stds  = grp.std().fillna(0)
        xs = means.index.values
        ax.plot(xs, means.values,
                marker=style["marker"], color=style["color"],
                linewidth=2, markersize=7, label=style["label"])
        ax.fill_between(xs,
                         means.values - stds.values,
                         means.values + stds.values,
                         color=style["color"], alpha=0.15)

    ax.set_xlabel(x_label)
    ax.set_ylabel(r"$\log_{10}(\|Ax - b\|_2)$")
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

In [ ]:
# ── Residual vs Condition Number ──────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(7, 5))

sub_cond = df_perf[np.isclose(df_perf["n"], FIXED_N) & np.isclose(df_perf["sparsity"], FIXED_SPARSITY)]
plot_residual_sweep(sub_cond, "cond_number",
                    r"Condition number $\kappa$",
                    "sparsity", FIXED_SPARSITY, ax)

ax.set_title(f"Residual vs Condition Number  ($n={FIXED_N}$, sparsity$={FIXED_SPARSITY}$)")
ax.set_xticks(COND_NUMBERS)
ax.set_xticklabels([str(int(k)) for k in COND_NUMBERS])
plt.tight_layout()
plt.savefig(f"../data/oracle_comparison_cond_number_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf")
plt.show()

In [ ]:
# ── Residual vs Sparsity ──────────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(7, 5))

sub_sp = df_perf[np.isclose(df_perf["n"], FIXED_N) & np.isclose(df_perf["cond_number"], FIXED_COND)]
plot_residual_sweep(sub_sp, "sparsity",
                    "Sparsity (fraction of zeros)",
                    "cond_number", FIXED_COND, ax)

ax.set_title(f"Residual vs Sparsity  ($n={FIXED_N}$, $\\kappa={int(FIXED_COND)}$)")
ax.set_xticks(SPARSITIES)
ax.set_xticklabels([f"{s:.1f}" for s in SPARSITIES])
plt.tight_layout()
plt.savefig(f"../data/oracle_comparison_sparsity_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf")
plt.show()

In [ ]:
# ── Residual vs Problem Size ──────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(7, 5))

sub_size = df_perf[
    np.isclose(df_perf["cond_number"], FIXED_COND) &
    np.isclose(df_perf["sparsity"],    FIXED_SPARSITY) &
    df_perf["n"].isin(PERF_SIZES)
]
plot_residual_sweep(sub_size, "n",
                    "Problem size $n$",
                    "cond_number", FIXED_COND, ax)

ax.set_title(f"Residual vs Problem Size  ($\\kappa={int(FIXED_COND)}$, sparsity$={FIXED_SPARSITY}$)")
ax.set_xticks(PERF_SIZES)
ax.set_xticklabels([str(n) for n in PERF_SIZES])
plt.tight_layout()
plt.savefig(f"../data/oracle_comparison_size_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf")
plt.show()

---
## Part 3 — Head-to-Head & Summary Analysis

In [ ]:
# ── Head-to-head scatter: hybrid residual vs quantum residual ─────────────────
# Each point is one (n, κ, sparsity, seed) instance

df_h = df_perf[df_perf["oracle"] == "hybrid"][["n","cond_number","sparsity","seed","residual"]].rename(columns={"residual": "res_hybrid"})
df_q = df_perf[df_perf["oracle"] == "quantum"][["n","cond_number","sparsity","seed","residual"]].rename(columns={"residual": "res_quantum"})
df_hh = df_h.merge(df_q, on=["n","cond_number","sparsity","seed"]).dropna()

fig, ax = plt.subplots(figsize=(7, 6))
sc = ax.scatter(
    np.log10(df_hh["res_hybrid"].clip(1e-10)),
    np.log10(df_hh["res_quantum"].clip(1e-10)),
    c=np.log10(df_hh["cond_number"]),
    cmap="viridis", alpha=0.75, s=60, edgecolors="none",
)
cbar = fig.colorbar(sc, ax=ax)
cbar.set_label(r"$\log_{10}(\kappa)$")

# Diagonal: equal performance
lims = [min(ax.get_xlim()[0], ax.get_ylim()[0]),
        max(ax.get_xlim()[1], ax.get_ylim()[1])]
ax.plot(lims, lims, "k--", linewidth=1, alpha=0.5, label="Equal performance")
ax.set_xlim(lims); ax.set_ylim(lims)

ax.set_xlabel(r"Hybrid HHL $\log_{10}(\|Ax-b\|_2)$")
ax.set_ylabel(r"Quantum HHL $\log_{10}(\|Ax-b\|_2)$")
ax.set_title("Head-to-Head Residual: Hybrid vs Quantum Oracle\n(points above diagonal → quantum worse)")
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f"../data/oracle_comparison_head2head_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf")
plt.show()

pct_hybrid_wins = 100 * (df_hh["res_hybrid"] < df_hh["res_quantum"]).mean()
print(f"Hybrid oracle achieves lower residual in {pct_hybrid_wins:.1f}% of instances")
print(f"Quantum oracle achieves lower residual in {100-pct_hybrid_wins:.1f}% of instances")

In [ ]:
# ── Summary heatmap: median residual across (κ, sparsity) for each oracle ─────

sub_hm = df_perf[np.isclose(df_perf["n"], FIXED_N) & df_perf["success"]].copy()
sub_hm["log_residual"] = np.log10(sub_hm["residual"].clip(1e-10))

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for ax, oracle_key in zip(axes[:2], ORACLE_MAP):
    piv = (
        sub_hm[sub_hm["oracle"] == oracle_key]
        .groupby(["cond_number", "sparsity"])["log_residual"]
        .median()
        .unstack("sparsity")
    )
    im = ax.imshow(piv.values, aspect="auto", origin="lower", cmap="RdYlGn_r",
                   vmin=-3, vmax=0)
    ax.set_xticks(range(len(piv.columns)))
    ax.set_xticklabels([f"{s:.1f}" for s in piv.columns])
    ax.set_yticks(range(len(piv.index)))
    ax.set_yticklabels([str(int(k)) for k in piv.index])
    ax.set_xlabel("Sparsity")
    ax.set_ylabel(r"Condition number $\kappa$")
    ax.set_title(f"{ORACLE_STYLE[oracle_key]['label']}\n" r"Median $\log_{10}(\|Ax-b\|_2)$")
    for i in range(len(piv.index)):
        for j in range(len(piv.columns)):
            ax.text(j, i, f"{piv.values[i,j]:.2f}", ha="center", va="center", fontsize=9)
    fig.colorbar(im, ax=ax, shrink=0.8)

# Difference heatmap: hybrid - quantum (negative = hybrid better)
ax = axes[2]
hm_hybrid  = sub_hm[sub_hm["oracle"]=="hybrid"].groupby(["cond_number","sparsity"])["log_residual"].median().unstack()
hm_quantum = sub_hm[sub_hm["oracle"]=="quantum"].groupby(["cond_number","sparsity"])["log_residual"].median().unstack()
diff = hm_hybrid - hm_quantum  # < 0 means hybrid better
im2 = ax.imshow(diff.values, aspect="auto", origin="lower", cmap="RdBu",
                vmin=-np.nanmax(np.abs(diff.values)), vmax=np.nanmax(np.abs(diff.values)))
ax.set_xticks(range(len(diff.columns)))
ax.set_xticklabels([f"{s:.1f}" for s in diff.columns])
ax.set_yticks(range(len(diff.index)))
ax.set_yticklabels([str(int(k)) for k in diff.index])
ax.set_xlabel("Sparsity")
ax.set_ylabel(r"Condition number $\kappa$")
ax.set_title("Difference (Hybrid − Quantum)\nBlue = Hybrid better, Red = Quantum better")
for i in range(len(diff.index)):
    for j in range(len(diff.columns)):
        ax.text(j, i, f"{diff.values[i,j]:+.2f}", ha="center", va="center", fontsize=9)
fig.colorbar(im2, ax=ax, shrink=0.8)

fig.suptitle(f"Residual Heatmap ($n={FIXED_N}$, median over {len(SEEDS)} seeds)",
             fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
plt.savefig(f"../data/oracle_comparison_heatmap_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf")
plt.show()

In [ ]:
# ── Resource efficiency: residual per unit of circuit depth ───────────────────
# Lower is better: cheaper circuit for the same residual

df_eff = df_perf[df_perf["success"]].copy()
df_eff["log_residual"] = np.log10(df_eff["residual"].clip(1e-10))

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

for ax, x_col, x_label in [
    (axes[0], "cond_number", r"Condition number $\kappa$"),
    (axes[1], "n",           "Problem size $n$"),
]:
    filter_col = "n" if x_col == "cond_number" else "cond_number"
    filter_val = FIXED_N if x_col == "cond_number" else FIXED_COND
    sub = df_eff[np.isclose(df_eff[filter_col], filter_val) & np.isclose(df_eff["sparsity"], FIXED_SPARSITY)]

    for oracle_key, style in ORACLE_STYLE.items():
        grp = sub[sub["oracle"] == oracle_key].groupby(x_col)
        means_res = grp["log_residual"].mean()

        ax.plot(means_res.index, means_res.values,
                marker=style["marker"], color=style["color"],
                linewidth=2, markersize=7, label=style["label"])

    ax.set_xlabel(x_label)
    ax.set_ylabel(r"$\log_{10}(\|Ax-b\|_2)$")
    ax.legend(fontsize=9, loc="upper left")
    ax.grid(True, alpha=0.3)

    if x_col == "cond_number":
        ax.set_xticks(COND_NUMBERS)
        ax.set_xticklabels([str(int(k)) for k in COND_NUMBERS])
        ax.set_title(f"Residual vs $\\kappa$  ($n={FIXED_N}$)")
    else:
        ax.set_xticks(PERF_SIZES)
        ax.set_xticklabels([str(n) for n in PERF_SIZES])
        ax.set_title(f"Residual vs $n$  ($\\kappa={int(FIXED_COND)}$)")

fig.suptitle("Solution Quality Comparison (Hybrid vs Quantum Oracle)",
             fontsize=14, fontweight="bold")
plt.tight_layout()
plt.savefig(f"../data/oracle_comparison_quality_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf")
plt.show()

In [ ]:
# ── Numerical summary table ───────────────────────────────────────────────────

summary = (
    df_perf[df_perf["success"]]
    .groupby("oracle")["residual"]
    .agg(["mean", "median", "std", "min", "max"])
    .rename(index={"hybrid": "Hybrid HHL", "quantum": "Quantum HHL"})
)
summary.columns = ["Mean residual", "Median residual", "Std", "Min", "Max"]
print("Performance summary across all problem instances:")
summary.round(5)

res_summary = (
    df_res.groupby("oracle")[["tr_depth", "tr_2q_gates"]]
    .agg(["mean", "max"])
)
print("\nResource summary across all problem sizes:")
res_summary.round(0)